<a href="https://colab.research.google.com/github/tusharastirmind-commits/in_class_exercises_2-3/blob/main/In_class_exercises_4_Text_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [10]:
# Setup (run this cell first)
import re
import pandas as pd
from google.colab import files

pd


<module 'pandas' from '/usr/local/lib/python3.12/dist-packages/pandas/__init__.py'>

## Q1 (1 point) — Answer below

In [11]:
# Q1 — ANSWER CELL
# 1. Trigger file upload
uploaded = files.upload()

# 2. Get the filename dynamically
file_name = list(uploaded.keys())[0]

# TODO: load the TAB-separated file into df
# Hint: pd.read_csv(FILE_PATH, sep="\t")
df = pd.read_csv(file_name, sep="\t")

# TODO: print df.shape and df.head(3)
df.shape
df.head(3)

Saving support_messages.txt to support_messages.txt


,id,message
0,1,Hi!! My ORDER is late :( Order# ORD-1042. Ema...
1,2,Refund please!!! I was charged 2 times... invo...
2,3,"Great service, thanks! arrived in 2 days :)"


## Q2 (3 points) — Answer below

In [13]:
# Q2 — ANSWER CELL
# TODO: create word_count, char_count, num_count, upper_word_count
# Hint for digits: df["message"].str.count(r"\d")
# Hint for ALL-CAPS words: count tokens where token.isupper()

# TODO: display/print the full DataFrame

# -----------------------------
# 2) Patterns for noisy/sensitive elements
# -----------------------------
EMAIL_RE = re.compile(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b")

# Basic US phone patterns (covers (469) 555-0182, 469-555-0182, 4695550182, etc.)
PHONE_RE = re.compile(
    r"\b(?:\+1[-.\s]?)?(?:\(?\d{3}\)?[-.\s]?)\d{3}[-.\s]?\d{4}\b"
)

# Order IDs like ORD-1042, ord-1060, "Order# ORD-1042", etc.
ORDER_RE = re.compile(r"\b(?:ord|ORD)[-_]?\d{3,10}\b")

# A simple address heuristic for this dataset:
# - street number + words + common street types + optional apt/unit + city/state/zip
ADDRESS_RE = re.compile(
    r"\b\d{1,6}\s+[A-Za-z0-9.\s]+(?:Rd|Road|St|Street|Ave|Avenue|Blvd|Boulevard|Ln|Lane|Dr|Drive|Ct|Court|Way|Pkwy|Parkway)\b"
    r"(?:\s+(?:Apt|Unit|Ste|Suite)\s*\d+)?"
    r"(?:,\s*[A-Za-z.\s]+,\s*[A-Z]{2}\s*\d{5})?",
    flags=re.IGNORECASE
)

# Common emoticons in the file: :) :( plus a few common variants
EMOTICON_RE = re.compile(r"(:\)|:\(|:D|;\)|:-\)|:-\()")

# Repeated punctuation like !!! or ???
REPEAT_PUNCT_RE = re.compile(r"([!?.,])\1{1,}")

# Abbreviations to expand
ABBREV_MAP = {
    r"\bpls\b": "please",
    r"\basap\b": "as soon as possible",
}




## Q3 (3 points) — Answer below

In [14]:
# Q3 — ANSWER CELL
# Option A (sklearn stopwords):
# from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# STOPWORDS = set(ENGLISH_STOP_WORDS)

# Option B (nltk stopwords):
# import nltk
# nltk.download("stopwords")
# from nltk.corpus import stopwords
# STOPWORDS = set(stopwords.words("english"))

def clean_text(text: str) -> str:
    # TODO: implement steps 1–4 in order
    return str(text)

# TODO: create df["clean"] using clean_text
# TODO: print original and clean for id=1 and id=4
# -----------------------------
# 3) Detection / counting (answers: "what noisy/sensitive data are present?")
# -----------------------------
def count_matches(series: pd.Series, regex: re.Pattern) -> int:
    return int(series.apply(lambda x: len(regex.findall(x))).sum())

def any_upper_lower_inconsistency(text: str) -> bool:
    # A simple signal: if there are mixed-case "ord/ORD" patterns or lots of all-caps words
    has_lower_ord = bool(re.search(r"\bord[-_]\d+\b", text))
    has_upper_ord = bool(re.search(r"\bORD[-_]\d+\b", text))
    has_all_caps_word = bool(re.search(r"\b[A-Z]{3,}\b", text))
    return (has_lower_ord and has_upper_ord) or has_all_caps_word

def has_informal_abbrev(text: str) -> bool:
    return bool(re.search(r"\bpls\b|\basap\b", text, flags=re.IGNORECASE))

noisy_summary = {
    "emails_found": count_matches(df["message"], EMAIL_RE),
    "phones_found": count_matches(df["message"], PHONE_RE),
    "order_ids_found": count_matches(df["message"], ORDER_RE),
    "addresses_found": count_matches(df["message"], ADDRESS_RE),
    "emoticons_found": count_matches(df["message"], EMOTICON_RE),
    "repeated_punctuation_found": count_matches(df["message"], REPEAT_PUNCT_RE),
    "rows_with_case_inconsistency_or_allcaps": int(df["message"].apply(any_upper_lower_inconsistency).sum()),
    "rows_with_informal_abbrev": int(df["message"].apply(has_informal_abbrev).sum()),
}

print("\n--- Noisy/Sensitive Data Summary ---")
for k, v in noisy_summary.items():
    print(f"{k}: {v}")





--- Noisy/Sensitive Data Summary ---
emails_found: 2
phones_found: 1
order_ids_found: 5
addresses_found: 3
emoticons_found: 2
repeated_punctuation_found: 7
rows_with_case_inconsistency_or_allcaps: 4
rows_with_informal_abbrev: 2


## Q4 (2 points) — Answer below

In [16]:
# Q4 — ANSWER CELL
# order_id pattern: r"ORD-\d{4}" with re.IGNORECASE
# email pattern (simple): r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"

# TODO: create df["order_id"] and df["email"]
# TODO: print/display df[["id", "order_id", "email"]]

# 4) Cleaning pipeline (answers: "what preprocessing steps should be applied?")
# -----------------------------
def expand_abbreviations(text: str) -> str:
    for pat, repl in ABBREV_MAP.items():
        text = re.sub(pat, repl, text, flags=re.IGNORECASE)
    return text

def clean_message(
    text: str,
    mask_emails: bool = True,
    mask_phones: bool = True,
    mask_addresses: bool = True,
    mask_order_ids: bool = True,
    remove_emoticons: bool = True,
    normalize_punct: bool = True,
    remove_special_chars: bool = True,
    lowercase: bool = True
) -> str:
    # 1) Normalize whitespace early
    text = text.strip()

    # 2) Expand abbreviations (pls/asap)
    text = expand_abbreviations(text)

    # 3) Mask PII / IDs
    if mask_emails:
        text = EMAIL_RE.sub("EMAIL", text)
    if mask_phones:
        text = PHONE_RE.sub("PHONE", text)
    if mask_addresses:
        text = ADDRESS_RE.sub("ADDRESS", text)
    if mask_order_ids:
        text = ORDER_RE.sub("ORDER_ID", text)

    # 4) Remove emoticons (optional)
    if remove_emoticons:
        text = EMOTICON_RE.sub(" ", text)

    # 5) Normalize repeated punctuation (!!! -> !, ??? -> ?)
    if normalize_punct:
        text = REPEAT_PUNCT_RE.sub(r"\1", text)

    # 6) Optionally remove special characters while keeping useful tokens
    if remove_special_chars:
        # Keep letters/numbers/spaces and a small set of punctuation that can help meaning
        # Then later collapse punctuation spacing
        text = re.sub(r"[^A-Za-z0-9\s.,!?'-]", " ", text)

    # 7) Lowercase (usually helps)
    if lowercase:
        text = text.lower()

    # 8) Collapse extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


# Apply cleaning
df["cleaned"] = df["message"].apply(clean_message)

print("\n--- Before/After Examples ---")
for i, row in df.head(8).iterrows():
    print(f"\nID {row.get('id', i+1)}")
    print("ORIG:", row["message"])
    print("CLEAN:", row["cleaned"])







--- Before/After Examples ---

ID 1
ORIG: Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com
CLEAN: hi! my order is late order order id. email me at email

ID 2
ORIG: Refund please!!! I was charged 2 times... invoice ORD-1042 and ORD-1043
CLEAN: refund please! i was charged address-address-1043

ID 3
ORIG: Great service, thanks! arrived in 2 days :)
CLEAN: great service, thanks! arrived in 2 days

ID 4
ORIG: WHY is my package DAMAGED??? tracking says delivered...
CLEAN: why is my package damaged? tracking says delivered.

ID 5
ORIG: Need to change address: 7421 Frankford Rd Apt 1232, Dallas TX 75252
CLEAN: need to change address address, dallas tx 75252

ID 6
ORIG: Support ticket: ORD-1050. Call me at (469) 555-0182 ASAP!!
CLEAN: support ticket order id. call me at phone as soon as possible!

ID 7
ORIG: I can’t login— password reset link not working. email: mehri.sattari@unt.edu
CLEAN: i can t login password reset link not working. email email

ID 8
ORIG: Item m

## Q5 (1 point) — Answer below

In [17]:
# Q5 — ANSWER CELL
# Hint: from sklearn.feature_extraction.text import TfidfVectorizer
# 1) fit TF-IDF on df["clean"]
# 2) compute average TF-IDF per term across documents
# 3) print top 5 terms + their average scores

# TODO
# -----------------------------
# 5) (Optional) Stopword removal helper
# -----------------------------
# Stopword removal depends on your task. For intent classification you might keep stopwords.
# If you want to remove stopwords anyway, here is a minimal example.
DEFAULT_STOPWORDS = {
    "i","me","my","is","the","a","an","and","or","to","in","on","at","of","for","it","this","that","was","were",
    "be","been","am","are","as","but","so","we","you","your","they","them","he","she","him","her","with","from"
}

def remove_stopwords(text: str, stopwords: set = DEFAULT_STOPWORDS) -> str:
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords]
    return " ".join(tokens)

# Example: create an additional column without stopwords
df["cleaned_no_stopwords"] = df["cleaned"].apply(remove_stopwords)

print("\n--- Cleaned (no stopwords) Examples ---")
print(df[["id","cleaned","cleaned_no_stopwords"]].head(5))


# -----------------------------
# 6) Save cleaned output (optional)
# -----------------------------
OUT_PATH = "/mnt/data/support_messages_cleaned.csv"
df.to_csv(OUT_PATH, index=False)
print(f"\nSaved cleaned dataset to: {OUT_PATH}")



--- Cleaned (no stopwords) Examples ---
   id                                            cleaned  \
0   1  hi! my order is late order order id. email me ...   
1   2  refund please! i was charged address-address-1043   
2   3           great service, thanks! arrived in 2 days   
3   4  why is my package damaged? tracking says deliv...   
4   5    need to change address address, dallas tx 75252   

                            cleaned_no_stopwords  
0     hi! order late order order id. email email  
1    refund please! charged address-address-1043  
2          great service, thanks! arrived 2 days  
3  why package damaged? tracking says delivered.  
4   need change address address, dallas tx 75252  


OSError: Cannot save file into a non-existent directory: '/mnt/data'

## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
